This script is used to create the schema and tables, and upload staged data to database.

In [45]:
import pandas as pd

from database_utils import db_connect
from constants import *

In [46]:
# try to establish the connection
db_conn, db_cur = db_connect()
# verify the connection
db_cur.execute('SELECT version()')
db_version = db_cur.fetchone()
print(db_version)

Connecting to the PostgreSQL database...
('PostgreSQL 11.11 on amd64-portbld-freebsd11.4, compiled by FreeBSD clang version 10.0.0 (git@github.com:llvm/llvm-project.git llvmorg-10.0.0-0-gd32170dbd5b), 64-bit',)


In [47]:
# create the tables by executing init.sql
db_cur.execute(open("init.sql", "r").read())
db_conn.commit()

In [48]:
def insert_table_from_csv(file_path, table_name, cursor, conn):
    data = pd.read_csv(file_path)
    if "Country Code" in data.columns:
        data.drop(["Country Code"])
    if "Year" in data.columns:
        data.drop(["Year"])
    data = [tuple(x) for x in data.to_numpy()]
    print(data[0])
    template = ','.join(['%s'] * len(data))
    query = 'INSERT INTO ' + table_name +' VALUES {}'.format(template)
    cursor.execute(query, data)
    conn.commit()

In [49]:
# upload the dimension tables
db_cur.execute(f"SET search_path = {SCHEMA_NAME};")
# insert_table_from_csv(COUNTRY_DIMENSION_CSV, COUNTRY_DIMENSION, db_cur, db_conn)
insert_table_from_csv(YEAR_DIMENSION_CSV, YEAR_DIMENSION, db_cur, db_conn)

(0, 2005)


ProgrammingError: can't adapt type 'numpy.int64'

In [ ]:
# close the cursor
if db_cur is not None:
    db_cur.close()

# close the connection
if db_conn is not None:
    db_conn.close()